In [6]:
# Python API for IBM Cloud
# https://cloud.ibm.com/docs/services/cloud-object-storage/libraries?topic=cloud-object-storage-python
import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values
COS_ENDPOINT = "https://s3.us-east.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
COS_API_KEY_ID = "FkhRf4C1KLzuGcbctz16L33TefTcCzpGrBRoLiihMRtJ" # eg "W00YiRnLW4a3fTjMB-odB-2ySfTrFBIQQWanc--P3byk"
COS_AUTH_ENDPOINT = "https://iam.cloud.ibm.com/identity/token"
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/17820f0b43654f6293224ad886b08c6d:df6186d9-51fe-4acf-9516-b23e351782ad::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"

# Create resource
cos = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [7]:
# List Bucket Names
def get_buckets():
    print("Retrieving list of buckets")
    try:
        buckets = cos.buckets.all()
        for bucket in buckets:
            print("Bucket Name: {0}".format(bucket.name))
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to retrieve list buckets: {0}".format(e))

In [8]:
get_buckets()

Retrieving list of buckets
Bucket Name: cloud-object-storage-w251-hw3-faces
Bucket Name: cloud-object-storage-w251-ring-video


In [9]:
# Test file upload information
bucket_name = "cloud-object-storage-w251-ring-video"
upload_filename = "/Users/katayounborojerdi/Desktop/IMG_3521.jpg"
object_name = "test_upload.jpg"

In [10]:
# File Upload

COS_SERVICE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/17820f0b43654f6293224ad886b08c6d:df6186d9-51fe-4acf-9516-b23e351782ad:bucket:cloud-object-storage-w251-ring-video"

def upload_large_file(bucket_name, item_name, file_path):
    print("Starting large file upload for {0} to bucket: {1}".format(item_name, bucket_name))

    # set the chunk size to 5 MB
    part_size = 1024 * 1024 * 5

    # set threadhold to 5 MB
    file_threshold = 1024 * 1024 * 5

    # Create client connection
    cos_cli = ibm_boto3.client("s3",
        ibm_api_key_id=COS_API_KEY_ID,
        ibm_service_instance_id=COS_SERVICE_CRN,
        ibm_auth_endpoint=COS_AUTH_ENDPOINT,
        config=Config(signature_version="oauth"),
        endpoint_url=COS_ENDPOINT
    )

    # set the transfer threshold and chunk size in config settings
    transfer_config = ibm_boto3.s3.transfer.TransferConfig(
        multipart_threshold=file_threshold,
        multipart_chunksize=part_size
    )

    # create transfer manager
    transfer_mgr = ibm_boto3.s3.transfer.TransferManager(cos_cli, config=transfer_config)

    try:
        # initiate file upload
        future = transfer_mgr.upload(file_path, bucket_name, item_name)

        # wait for upload to complete
        future.result()

        print ("Large file upload complete!")
    except Exception as e:
        print("Unable to complete large file upload: {0}".format(e))
    finally:
        transfer_mgr.shutdown()



In [11]:
upload_large_file(bucket_name, object_name, upload_filename)

Starting large file upload for test_upload.jpg to bucket: cloud-object-storage-w251-ring-video
Large file upload complete!


In [12]:
def get_bucket_contents_v2(bucket_name, max_keys):
    print("Retrieving bucket contents from: {0}".format(bucket_name))
    try:
        # create client object
        cos_cli = ibm_boto3.client("s3",
            ibm_api_key_id=COS_API_KEY_ID,
            ibm_service_instance_id=COS_SERVICE_CRN,
            ibm_auth_endpoint=COS_AUTH_ENDPOINT,
            config=Config(signature_version="oauth"),
            endpoint_url=COS_ENDPOINT)

        more_results = True
        next_token = ""

        while (more_results):
            response = cos_cli.list_objects_v2(Bucket=bucket_name, MaxKeys=max_keys, ContinuationToken=next_token)
            files = response["Contents"]
            for file in files:
                print("Item: {0} ({1} bytes).".format(file["Key"], file["Size"]))

            if (response["IsTruncated"]):
                next_token = response["NextContinuationToken"]
                print("...More results in next batch!\n")
            else:
                more_results = False
                next_token = ""

    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to retrieve bucket contents: {0}".format(e))

In [13]:
get_bucket_contents_v2(bucket_name, 100)

Retrieving bucket contents from: cloud-object-storage-w251-ring-video
Item: 2020-02-28_08:35.mp4 (26176721 bytes).
Item: 2020-02-28_11:47.mp4 (28394994 bytes).
Item: 2020-02-28_11:48.mp4 (26247747 bytes).
Item: 2020-02-28_12:22.mp4 (26281752 bytes).
Item: 2020-02-28_12:29.mp4 (25405141 bytes).
Item: 2020-02-28_13:04.mp4 (27515544 bytes).
Item: 2020-02-28_13:20.mp4 (25743342 bytes).
Item: 2020-02-28_14:46.mp4 (25393728 bytes).
Item: 2020-02-28_14:51.mp4 (25234346 bytes).
Item: 2020-02-28_15:50.mp4 (25933091 bytes).
Item: 2020-02-28_16:24.mp4 (25327037 bytes).
Item: 2020-02-28_16:30.mp4 (27025344 bytes).
Item: 2020-02-28_16:49.mp4 (27198376 bytes).
Item: 2020-02-28_16:50.mp4 (26870803 bytes).
Item: 2020-02-28_16:52.mp4 (25723109 bytes).
Item: 2020-02-28_16:54.mp4 (26559788 bytes).
Item: 2020-02-28_16:56.mp4 (26549176 bytes).
Item: 2020-02-28_17:00.mp4 (26385470 bytes).
Item: 2020-02-28_17:02.mp4 (25878675 bytes).
Item: 2020-02-28_17:03.mp4 (24446360 bytes).
Item: 2020-02-28_17:05.mp4 (26

In [14]:
def get_item(bucket_name, item_name):
    print("Retrieving item from bucket: {0}, key: {1}".format(bucket_name, item_name))
    try:
        file = cos.Object(bucket_name, item_name).get()
        print("File Contents: {0}".format(file["Body"].read()))
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to retrieve file contents: {0}".format(e))

In [17]:
item_name = "2020-02-28_08:35.mp4"

In [20]:
# Below cell should work if you start notebook with the following
# jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

In [18]:
test_video = get_item(bucket_name, item_name)

Retrieving item from bucket: cloud-object-storage-w251-ring-video, key: 2020-02-28_08:35.mp4


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [53]:
import json
import getpass
from pathlib import Path

from ring_doorbell import Ring, Auth
from oauthlib.oauth2 import MissingTokenError


cache_file = Path("ring_token.cache")


def token_updated(token):
    cache_file.write_text(json.dumps(token))


def otp_callback():
    auth_code = input("2FA code: ")
    return auth_code


def ring_first_login():
    if cache_file.is_file():
        auth = Auth("MyProject/1.0", json.loads(cache_file.read_text()), token_updated)
    else:
        username = input("Username: ")
        password = getpass.getpass("Password: ")
        auth = Auth("MyProject/1.0", None, token_updated)
        try:
            auth.fetch_token(username, password)
        except MissingTokenError:
            auth.fetch_token(username, password, otp_callback())

    ring = Ring(auth)
    ring.update_data()

    devices = ring.devices()
    print(devices)

In [54]:
ring_first_login()

Username: Brandon.fennema@gmail.com
Password: ········
2FA code: 298744
{'stickup_cams': [], 'chimes': [], 'doorbots': [<RingDoorBell: Front Door>], 'authorized_doorbots': []}


In [76]:
# Generate a new token

auth = Auth("MyProject/1.0", json.loads(cache_file.read_text()), token_updated)
ring = Ring(auth)
ring.update_data()
devices = ring.devices()

pprint(ring.session['profile'])

{'account_type': None,
 'app_brand': 'ring',
 'authentication_token': '_3yy5WfBoexwdzQdNSDS',
 'country': 'US',
 'created_at': '2019-12-29T18:03:54Z',
 'email': 'brandon.fennema@gmail.com',
 'explorer_program_terms': None,
 'features': {'2fa_dur_reg_enabled': False,
              'accelerated_alarm_enabled': True,
              'accessories_upsell_1_0_enabled': False,
              'adaptive_video_play_enabled': True,
              'advanced_motion_detection_device_types': ['doorbell_portal',
                                                         'cocoa_camera',
                                                         'cocoa_doorbell',
                                                         'doorbell_scallop',
                                                         'doorbell_scallop_lite'],
              'advanced_motion_detection_human_only_mode_enabled': False,
              'advanced_motion_zones_device_types': [],
              'advanced_motion_zones_enabled': False,
          

In [59]:
devices = ring.devices()
doorbell = devices['doorbots'][0]
doorbell.recording_download(
    doorbell.history(limit=100, kind='ding')[0]['id'],
                     filename='last_ding.mp4',
                     override=True)

True

In [60]:
devices = ring.devices()
for doorbell in devices['doorbots']:

    # listing the last 15 events of any kind
    for event in doorbell.history(limit=15):
        print('ID:       %s' % event['id'])
        print('Kind:     %s' % event['kind'])
        print('Answered: %s' % event['answered'])
        print('When:     %s' % event['created_at'])
        print('--' * 50)
    

ID:       6798673830757708585
Kind:     motion
Answered: False
When:     2020-02-29 01:26:44+00:00
----------------------------------------------------------------------------------------------------
ID:       6798672920224641833
Kind:     motion
Answered: False
When:     2020-02-29 01:23:12+00:00
----------------------------------------------------------------------------------------------------
ID:       6798672331814122281
Kind:     motion
Answered: False
When:     2020-02-29 01:20:55+00:00
----------------------------------------------------------------------------------------------------
ID:       6798671670389158697
Kind:     motion
Answered: False
When:     2020-02-29 01:18:21+00:00
----------------------------------------------------------------------------------------------------
ID:       6798671154993083177
Kind:     motion
Answered: False
When:     2020-02-29 01:16:21+00:00
----------------------------------------------------------------------------------------------------


In [71]:
from ring_doorbell import Ring
import urllib.request
from datetime import datetime, timedelta, date
import pytz
import configparser
import logging
import os

pst = pytz.timezone('US/Pacific')

In [66]:
print(video_history)

[{'id': 6798673830757708585, 'created_at': datetime.datetime(2020, 2, 29, 1, 26, 44, tzinfo=<UTC>), 'answered': False, 'events': [], 'kind': 'motion', 'favorite': False, 'snapshot_url': '', 'recording': {'status': 'ready'}, 'duration': 61.0, 'cv_properties': {'person_detected': None, 'stream_broken': None, 'detection_type': None}}, {'id': 6798672920224641833, 'created_at': datetime.datetime(2020, 2, 29, 1, 23, 12, tzinfo=<UTC>), 'answered': False, 'events': [], 'kind': 'motion', 'favorite': False, 'snapshot_url': '', 'recording': {'status': 'ready'}, 'duration': 61.0, 'cv_properties': {'person_detected': None, 'stream_broken': None, 'detection_type': None}}]


In [78]:
# bucket_name = "cloud-object-storage-w251-ring-video"
# upload_filename = "/Users/katayounborojerdi/Desktop/IMG_3521.jpg"
# object_name = "test_upload.jpg"

video_history = doorbell.history(limit = 30)
for video in video_history:
    print(video['id'])
    filename = video['created_at'].astimezone(pst).strftime("%Y-%m-%d_%H:%M") + ".mp4"
    upload_filename = "/Users/katayounborojerdi/" + filename
    print(filename)

    doorbell.recording_download(video['id'],filename,override=True)
    
    upload_large_file(bucket_name, filename, upload_filename)
    
    os.remove(filename)

6798673830757708585
2020-02-28_17:26.mp4
Starting large file upload for 2020-02-28_17:26.mp4 to bucket: cloud-object-storage-w251-ring-video
Large file upload complete!
6798672920224641833
2020-02-28_17:23.mp4
Starting large file upload for 2020-02-28_17:23.mp4 to bucket: cloud-object-storage-w251-ring-video
Large file upload complete!
6798672331814122281
2020-02-28_17:20.mp4
Starting large file upload for 2020-02-28_17:20.mp4 to bucket: cloud-object-storage-w251-ring-video
Large file upload complete!
6798671670389158697
2020-02-28_17:18.mp4
Starting large file upload for 2020-02-28_17:18.mp4 to bucket: cloud-object-storage-w251-ring-video
Large file upload complete!
6798671154993083177
2020-02-28_17:16.mp4
Starting large file upload for 2020-02-28_17:16.mp4 to bucket: cloud-object-storage-w251-ring-video
Large file upload complete!
6798670338949296937
2020-02-28_17:13.mp4
Starting large file upload for 2020-02-28_17:13.mp4 to bucket: cloud-object-storage-w251-ring-video
Large file upl